In [1]:
import pandas as pd
import requests
from datetime import datetime
import matplotlib.pyplot as plt
import math
import os
from pathlib import Path


In [5]:
os.chdir(Path(os.getcwd()).parent)

from utils.utils import getPublicContractsGPT, getPublicContractsLLM, getTickerGPT, getTickerLlma

In [6]:
contracts = pd.read_csv("data/contractsRaw.csv")
contracts_test = contracts[contracts["Start Date"].str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)]
contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])
contracts_test = contracts_test.sort_values(by="Start Date")

/tmp/ipykernel_1043765/924723225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])


In [7]:
client = OpenAI(api_key="")

In [8]:
filtered_df = contracts_test[
    (contracts_test["Start Date"] >= pd.Timestamp("2013-01-01")) & 
    (contracts_test["Start Date"] <= pd.Timestamp("2013-03-31"))
]
filtered_df

,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
2371,15094023,FA880813C0002,"KRATOS TECHNOLOGY & TRAINING SOLUTIONS, INC.",309770036.79,8035252.37,COMMAND AND CONTROL - CONSOLIDATED (CCS-C) PRO...,DEFINITIVE CONTRACT,"['C', 'E', 'N', 'Q']",-2966766.49,3107361.49,...,NaN,Department of Defense,Department of the Air Force,2013-01-01,2024-03-31,919f88f6-b5cf-feac-7300-0fc4d72b79a1-C,NaN,1173,department-of-defense,CONT_AWD_FA880813C0002_9700_-NONE-_-NONE-
4349,35993711,V764C35004,MCKESSON CORPORATION,164165493.76,NaN,EXPRESS REPORT + PHARMACEUTICALS,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Veterans Affairs,Department of Veterans Affairs,2013-01-01,2013-01-31,b54de748-ee64-a1e6-546e-e7007b054797-C,NaN,561,department-of-veterans-affairs,CONT_AWD_V764C35004_3600_VA797P12D0001_3600
15779,28087259,SD1F,NORTHROP GRUMMAN SYSTEMS CORP,43841067.0,NaN,CY13 PERFORMANCE-BASED LOGISTICS (PBL) B-2 SOF...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2013-01-01,2018-07-31,a4e18d9e-ec75-2c26-9475-543f727fdbe7-C,NaN,1173,department-of-defense,CONT_AWD_SD1F_9700_F3365799D0028_9700
11277,24127666,HSCGG313JPWX002,GUIDEHOUSE DIGITAL LLC,60938643.24,NaN,IGF::OT::IGF CORE TECHNOLOGIES CONTRACT (CTS)...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Homeland Security,U.S. Coast Guard,2013-01-01,2015-05-31,7d6153d4-4d5e-6de7-cd4c-10cd4c355b26-C,NaN,766.0,department-of-homeland-security,CONT_AWD_HSCGG313JPWX002_7008_HSCGG311DPWV888_...
20504,28087251,SD1E,NORTHROP GRUMMAN SYSTEMS CORP,33676608.16,NaN,CY13 B-2 PERFORMANCE BASED LOGISTICS (PBL) SUS...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Contract Management Agency,2013-01-01,2018-06-28,a4e18d9e-ec75-2c26-9475-543f727fdbe7-C,NaN,1173,department-of-defense,CONT_AWD_SD1E_9700_F3365799D0028_9700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19247,15306638,FY08,BAE SYSTEMS TECHNOLOGY SOLUTIONS & SERVICES INC.,35928916.06,NaN,IGF::OT::IGF IN-SERVICE ENGINEERING AND INTEGR...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Navy,2013-03-29,2017-03-31,2e141abe-fe8f-7ddc-e6a3-8bb2165836be-C,NaN,1173,department-of-defense,CONT_AWD_FY08_9700_N0017804D4018_9700
17146,37265128,VC15,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION,40346199.94,NaN,TASK AREA 10 - TEST AND EVALUATION TASK AREA...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Information Systems Agency,2013-03-29,2017-10-31,20af0683-f17d-1f8f-38ee-2f11bd124559-C,NaN,1173,department-of-defense,CONT_AWD_VC15_9700_HC102808D2025_9700
8045,13662058,DOLJ139E34447,CONDUENT FEDERAL SOLUTIONS LLC,85170609.4,0.0,IGF::CT::IGF - TO PROVIDE EFFICIENCY AND CONSI...,DEFINITIVE CONTRACT,NaN,NaN,NaN,...,NaN,Department of Labor,Office of the Assistant Secretary for Administ...,2013-03-30,2016-09-30,630587c2-c4d6-a701-ee6b-41b63048486b-C,NaN,267,department-of-labor,CONT_AWD_DOLJ139E34447_1605_-NONE-_-NONE-
25408,15471199,GGS8,PERATON ENTERPRISE SOLUTIONS LLC,27212078.28,NaN,IGF::OT::IGF OTHER ADP AND TELECOMMUNICATIONS SVS,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Navy,2013-03-31,2013-09-30,333909ba-42cf-a1ce-3959-1ede2882d7f6-C,NaN,1173,department-of-defense,CONT_AWD_GGS8_9700_N0003910D0010_9700


In [9]:
alphV = "https://www.alphavantage.co/query"

In [10]:
def setParams(ticker, itv, date):
    key = f'{currentKey}'
    params = {
        "function": "TIME_SERIES_INTRADAY", 
        "symbol" : ticker, 
        "interval" : itv, 
        "adjusted" : "false", 
        "month" : date, 
        "outputsize":"full", 
        "datatype" : "json", 
        "extended_hours": "false",
        "apikey": newkeys[key]
    }
    return params 

In [ ]:
def getHistoric(tik, interval, month):
    global keyCalls, currentKey
    historic = requests.get(alphV, params=setParams(tik, interval, month))
    print("API called")
    key = f"Time Series ({interval})"
    if key not in historic.json():
        return pd.DataFrame() 
    else: 
        df = pd.DataFrame.from_dict(historic.json()[key]).transpose().reset_index().rename(columns={'index':'date'})
        df.columns = df.columns.str.replace(r'^\d+\.\s*', '', regex=True)
        df['date'] = pd.to_datetime(df['date'])
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df.sort_values(by='date', ascending= True).reset_index(drop=True)

In [12]:
def getData(tik, interval, contractDate):
    date = datetime.strptime(contractDate, "%Y-%m-%d")
    pre = (date.replace(day=1) - pd.DateOffset(months=1)).strftime("%Y-%m")
    post = (date.replace(day=1) + pd.DateOffset(months=1)).strftime("%Y-%m")
    df1 = getHistoric(tik, interval, pre)
    if df1.empty:
        return pd.DataFrame()
    else: 
        df2 = getHistoric(tik, interval, date.strftime("%Y-%m"))
        df3 = getHistoric(tik, interval, post)
        return pd.concat([df1, df2, df3], ignore_index=True, sort=False)
    

In [13]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

def getTradingDay(contractDate):
    us_business_day = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    adjDate = contractDate if us_business_day.is_on_offset(contractDate) else contractDate + us_business_day
    return adjDate.strftime('%Y-%m-%d')

In [14]:
def getContractInterval(data, contractDate):
    return data[data['date'].dt.strftime('%Y-%m-%d').str.contains(getTradingDay(contractDate))]

In [15]:
def genGraph(tik, data, contractInterval):
    global graph 

    plt.figure(figsize=(20, 10))

    plt.plot(data.index, data['close'], label='Close Price')
    x_min = data['date'].min().strftime("%Y-%m-%d")
    x_max = data['date'].max().strftime("%Y-%m-%d")
    y_min = data['close'].min()
    y_max = data['close'].max()
    y_ticks = list(range(math.ceil(y_min -1), math.ceil(y_max + 1.5), math.ceil(abs(y_max - y_min) / 5)))
    plt.yticks(y_ticks)

    unique_dates = data['date'].dt.strftime('%Y-%m-%d').unique()
    tick_positions = [data[data['date'].dt.strftime('%Y-%m-%d') == date].index[0] for date in unique_dates]
    plt.xticks(ticks=tick_positions, labels=unique_dates, rotation=45, ha='right')
    plt.scatter(contractInterval.index, contractInterval['close'], color='red', label='Point of Interest')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title(f'{tik} close price between {x_min} to {x_max}')
    plt.grid()

    plt.tight_layout()
    plt.legend()
    outputFile = "/home/xikron/Projects/misc/graphs/"+tik+"_"+str(graph)+".pdf"
    graph+=1
    plt.savefig(outputFile, format='pdf')
    plt.close()
    

In [16]:
def graphTester(tik, interval, contractDate): 
    df = getData(tik, interval, contractDate)
    if df.empty: 
        print(tik+ " contract on " + contractDate + " failed output")
    else:
        genGraph(tik, df, getContractInterval(df, contractDate))

In [17]:
def processRow(row): 
    ticker = getTickerGPT(row['Recipient Name'])
    if ticker!="none": 
        print(ticker)
        print("contract date: " + row["Start Date"].strftime("%Y-%m-%d"))
        graphTester(ticker, "15min", row["Start Date"].strftime("%Y-%m-%d"))

In [18]:
def filterRow(row): 
    global tradedContracts
    ticker = getTickerGPT(row['Recipient Name'])
    features = []
    if ticker!="none": 
        features.append(ticker)
        features.append(str(row["Start Date"].strftime("%Y-%m-%d")))
        features.append(row["End Date"])
        features.append(row["Award Amount"])
        tradedContracts.loc[len(tradedContracts)] = features

In [ ]:
tradedContracts = pd.DataFrame()
tradedContracts["Company"] =[]
tradedContracts["Start Date"] = []
tradedContracts["End Date"] = []
tradedContracts["valuation"] = []

filtered_df.apply(filterRow, axis=1)

tradedContracts.to_csv("/home/xikron/Projects/misc/data/traded_contracts.csv")

In [71]:
dft = pd.read_csv("/home/xikron/Projects/misc/data/traded_contracts.csv").drop(columns="Unnamed: 0")
dft.head() 

,Company,Start Date,End Date,valuation
0,NAVI,2013-01-01,2013-12-31,1.080254e+08
1,RTX,2013-01-01,2017-06-30,3.927845e+07
2,GE,2013-01-01,2013-12-31,1.380385e+08
3,BA,2013-01-01,2015-12-31,6.470494e+07
4,BA,2013-01-01,2013-12-31,8.032353e+07


In [66]:
dft['Start Date'] = pd.to_datetime(dft['Start Date'])
dft = dft.sort_values(by='valuation', ascending=False).drop_duplicates(subset=['Company', 'Start Date'])
dft.to_csv("/home/xikron/Projects/misc/data/traded_contracts_unique.csv")

In [72]:
dft =pd.read_csv("/home/xikron/Projects/misc/data/traded_contracts.csv").drop(columns="Unnamed: 0")
dft.shape

(107, 4)

In [73]:
def graphRow(row): 
    print(currentKey)
    graphTester(row['Company'], "15min", row["Start Date"])

In [74]:
currentKey =5
graph =20

In [56]:
dft.apply(graphRow, axis=1)

4
API called
API called
API called


0    None
dtype: object